# Вебинар 3. Коллаборативная фильтрация. 
### Домашнее задание

### 1. Загрузка и обработка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('transaction_data.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# ДЗ

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

## Практика:

4) Поэкспериментировать с ALS (grid-search)

### 2. Grid-search c ALS

In [25]:
import itertools

In [8]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

c:\users\n.burmistrova\appdata\local\programs\python\python37-32\lib\site-packages\pandas-1.0.1-py3.7-win32.egg\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [73]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [74]:
def learning_als(user_item_matrix, params, result):
    model = AlternatingLeastSquares(**params)
    print(params)
    model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
              show_progress=True)
    result['temp_als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    pers_at_k = result.apply(lambda row: precision_at_k(row['temp_als'], row['actual']), axis=1).mean()
    return pers_at_k

In [75]:
def grid_search_ALS(user_item_matrix, param_grid, result):

    curves = []
    keys, values = zip(*param_grid.items())
    iteration = 0
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        
        pers_at_k = learning_als(user_item_matrix, params, result)

        curves.append({'params': params,
                       'pers_at_k': pers_at_k,
                       'iteration': iteration})
        iteration += 1
    return curves

In [76]:
param_grid_test = {'factors': [50, 100],
              'regularization': [0.001],
              'iterations': [10, 15],
              'num_threads': [4],
              'calculate_training_loss': [True]}

In [37]:
param_grid_final = {'factors': [50, 100, 150, 200],
              'regularization': [0.001, 0.0001, 0.005, 0.01, 0.1, 1],
              'iterations': [10, 15, 20, 25, 100],
              'num_threads': [4],
              'calculate_training_loss': [True]}

In [77]:
grid_search_results = grid_search_ALS(user_item_matrix, param_grid_final, result)

{'factors': 50, 'regularization': 0.001, 'iterations': 10, 'num_threads': 4, 'calculate_training_loss': True}


  0%|          | 0/10 [00:00<?, ?it/s]

{'factors': 50, 'regularization': 0.001, 'iterations': 15, 'num_threads': 4, 'calculate_training_loss': True}


  0%|          | 0/15 [00:00<?, ?it/s]

{'factors': 50, 'regularization': 0.001, 'iterations': 20, 'num_threads': 4, 'calculate_training_loss': True}


...

  0%|          | 0/20 [00:00<?, ?it/s]

{'factors': 200, 'regularization': 1, 'iterations': 25, 'num_threads': 4, 'calculate_training_loss': True}


  0%|          | 0/25 [00:00<?, ?it/s]

{'factors': 200, 'regularization': 1, 'iterations': 100, 'num_threads': 4, 'calculate_training_loss': True}


  0%|          | 0/100 [00:00<?, ?it/s]

In [80]:
best_grid_search_results = sorted(grid_search_results, key=lambda x: x['pers_at_k'], reverse=True)
best_grid_search_results

[{'params': {'factors': 50,
   'regularization': 0.01,
   'iterations': 10,
   'num_threads': 4,
   'calculate_training_loss': True},
  'pers_at_k': 0.16993143976493633,
  'iteration': 15},
 {'params': {'factors': 50,
   'regularization': 0.1,
   'iterations': 15,
   'num_threads': 4,
   'calculate_training_loss': True},
  'pers_at_k': 0.16875612144955926,
  'iteration': 21},
 {'params': {'factors': 50,
   'regularization': 1,
   'iterations': 25,
   'num_threads': 4,
   'calculate_training_loss': True},
  'pers_at_k': 0.1685602350636631,
  'iteration': 28},
 {'params': {'factors': 50,
   'regularization': 1,
   'iterations': 10,
   'num_threads': 4,
   'calculate_training_loss': True},
  'pers_at_k': 0.1683643486777669,
  'iteration': 25},
 {'params': {'factors': 50,
   'regularization': 0.005,
   'iterations': 15,
   'num_threads': 4,
   'calculate_training_loss': True},
  'pers_at_k': 0.1682664054848188,
  'iteration': 11},
 {'params': {'factors': 50,
   'regularization': 0.1,
   'i

In [81]:
best_grid_search_results[0]

{'params': {'factors': 50,
  'regularization': 0.01,
  'iterations': 10,
  'num_threads': 4,
  'calculate_training_loss': True},
 'pers_at_k': 0.16993143976493633,
 'iteration': 15}

**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

- А точно нужно сортировать по вероятности?
По вероятности чего? того, что пользователю понравится товар? 
Да, но также нужно учесть стоимость данного товара. Исключить, к примеру, очень дорогие товары, или отправлять их если пользователь недавно присматривался к похожим. Но чтобы такого товара было 1-2 максимум в рассылке.
- Какую метрику использовать?
Precision at k, money_precision_at_k, recall_at_k и money_recall_at_k.
- Сколько раз в неделю отправляем рассылку?
Надо посчитать косты и доход с рассылки. Но, думаю, не чаще 1 раза в неделю, чтобы не спамить.
- В какое время отправляем рассылку?
Могу судить только по себе, когда я проверяю личную почту. В один из этих моментов: утром, когда прихожу на работу (в пандемии время проверки сдвинулось), вечером перед уходом, иногда могу в обед. Если аудитория женская, то в другие промежутки домашние дела и работа.
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
Фильтрами, может быть где-то фиксировать кому мы что отправили.
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
Смотря на что тригеррит рассылка наверное. Если пользователь давно не проявлял активности, то лучше отправить разные, популярные у похожих пользователей. Если только что смотрел холодильники, отправить ему неразные, мне кажется, можно. Холодильники по акции. Разные - что они относятся к разной категории? Как добиться - Может быть разделить товары на категории, и из каждой категории выбрать подходящий товар, который больше всего понравится пользователю.